In [1]:
import requests
from bs4 import BeautifulSoup
import pymysql
import re

In [2]:
! pip install unidecode

In [2]:
def get_job_info(trabajo_link):
    """El input es un enlace que dirige a una única oferta de trabajo. Devuelve un tuple con la información más relevante: descripción del anuncio y del trabajo"""
    anuncio_list = []
    description_list = []
    import requests
    from bs4 import BeautifulSoup
    try:
        response = get_page(trabajo_link)
        result_page = BeautifulSoup(response.content,'lxml')
        anuncio_list = get_anuncio_list(result_page)
        description_list = get_job_description(result_page)
        empresa_name = get_name_empresa(result_page)
        output = unify_lists([empresa_name, anuncio_list, description_list])
        return output
    except:
        return None

In [3]:
def get_page(url):
    response = requests.get(url)
    if not response.status_code == 200:
        print("It didn't work")
    return response

In [4]:
def get_anuncio_list(result_page):
    '''This function recover the information concerning ID Empleo, Categoria, Ubicación, etc'''
    anuncio_list = []
    for info in result_page.find_all('div', class_='smallListingInfo'):
        if not 'Primera' in info.get_text(): #Exclude 'Primera Publicacion'
            if not 'Contrato' in info.get_text():
                anuncio_item = info.get_text()
                anuncio_item = anuncio_item[anuncio_item.find(':')+2:]
                anuncio_list.append(anuncio_item.replace('\n',''))  #Get items without \n
                #anuncio_list.append(info.get_text().replace('\n',''))
    return anuncio_list

In [5]:
def get_job_description(result_page):
    text = ''
    for paragraph in result_page.find_all('p'):
        if paragraph.text != "\n":
            text = text + paragraph.text
    text = clean_text(text)
    return text

In [6]:
def get_name_empresa(result_page):
    Enterprise_information = result_page.find('div', class_ = 'compProfileInfo')
    return Enterprise_information.find('strong').get_text()

In [7]:
def clean_text(text):
    import unidecode
    text = text.lower().replace('\n','') #Get items without \n
    text = re.sub("(\(\w+\))", '', text) #Delete parenthesis
    text = unidecode.unidecode(text) #Delete accents
    text = re.sub('[^A-Za-z0-9\s]','',text) #Delete anything but letters, numbers and spaces
    return text

In [8]:
def unify_lists(job_list):
    new_list = []
    new_list.append(job_list[0])
    for element in job_list[1]:
        new_list.append(element)
    new_list.append(job_list[2])
    return new_list

In [9]:
trabajo_link = "https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/695472/Analista-de-Tesorería.html?searchId=1545862649.53&page=1"
response = get_page(trabajo_link)
result_page = BeautifulSoup(response.content,'lxml')
anuncio_list = get_anuncio_list(result_page)
description_list = get_job_description(result_page)
empresa_name = get_name_empresa(result_page)
output = unify_lists([empresa_name, anuncio_list, description_list])

In [10]:
output

['NOSSAMODA SRL',
 '695472',
 'Santa Cruz de la Sierra, Bolivia',
 'Contabilidad y Economía ',
 'De Bs. 3.000 a Bs. 3.999 ',
 '28 Enero 2019',
 '27 Febrero 2019',
 'analista de tesoreria formacion titulado en auditoria financiera contaduria publica contaduria general o ramas afines experiencia y conocimientos  tres  anos de experiencia laboral en cargos de igual responsabilidad  elaboracion de conciliaciones bancarias y arqueos de caja elaboracion de flujos de caja gestion de pagos a proveedores locales y extranjeros seguimiento de cuentas pendientes de cobro y depositos de cobranzas del dia  preparacion de estados de cuentas por cobrar y pagar y control de emision de cheques registro contable de pagos a proveedores sueldos finiquitos etc  registro contable de cobros a clientes depositos transferencias etc  conocimiento en sistema contable sai   solidos manejo de windows office excel powerpoint outlook y word competencias  responsabilidad  organizacion  atencion a los detalles las pers

In [11]:
def trabajopolis(job_description):
    """ A partir de una (o más) palabras claves  """
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    for job in job_description:
        #url = "http://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D="+ job + "&page=1&listings_per_page=1000"
        url = "https://www.trabajopolis.bo/search-results-jobs/?action=search&listing_type%5Bequal%5D=Job&listings_order_by=desc&JobCategory%5Bmulti_like%5D%5B%5D=&CityBolivia%5Bmulti_like%5D%5B%5D=&PostedWithin%5Bmulti_like%5D%5B%5D=&EmploymentType%5Bmulti_like%5D%5B%5D=&keywords%5Bany_words%5D=cajero&page=1&listings_per_page=10"

        response = requests.get(url)
        if response.status_code == 200:
            print("The request response cycle was successful")
            results_page = BeautifulSoup(response.content,'lxml')
            #trabajo_list = list()
            trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
            for anuncio in trabajos:
                trabajo_link = anuncio.find('a').get('href')
                trabajo_name = anuncio.find('b').get_text()
                print(trabajo_link)
             #   trabajo_list.append((trabajo_name,trabajo_link))
            #for trabajo in trabajo_list:
                output = get_job_info(trabajo_link)
                trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
        else:
            print("The request response cycle work didn't work")
        time.sleep(10) 
    return trabajos_list

In [12]:
def trabajopolis2():
    trabajos_list = []
    import requests
    from bs4 import BeautifulSoup
    url = "https://www.trabajopolis.bo/search-results-jobs/?searchId=1546400657&action=search&page=1&listings_per_page=1500"
    response = requests.get(url)
    if response.status_code == 200:
        print("The request response cycle was successful")
        results_page = BeautifulSoup(response.content,'lxml')
        trabajos = results_page.findAll('td',{"class":"anuncio-estandar-content"})
        for anuncio in trabajos:
            trabajo_link = anuncio.find('a').get('href')
            trabajo_name = anuncio.find('b').get_text()
            print(trabajo_name)
            output = get_job_info(trabajo_link)
            trabajos_list.append((output,trabajo_name,trabajo_link))
            #I need to create a list that save all the extracted data
    else:
        print("The request response cycle work didn't work")
    return trabajos_list

In [13]:
Results = trabajopolis2()

The request response cycle was successful
Contador(a) de Costos
Jefe Comercial
Ejecutivo(a) de Siniestros - Santa Cruz
Ejecutivo(a) de Ventas - Segmento Consumo Masivo
Responsable Comercial - Cochabamba
Asesores(as) de Ventas
Auxiliar Comercial
Jefe de Distribución - Cochabamba
Gerente de Zona - Trinidad
Responsable Comercial - La Paz
Electricista - Temporal - Santa Cruz
Ejecutivo(a) de Ventas - Segmento Consumo Masivo
Ejecutivo(a) de Ventas - Línea Baterías
Auxiliar de Servicio Técnico - Línea Baterías
Trade Marketing Manager - Santa Cruz
Vendedor(a)
Apoyo Contable - La Paz
Mecánico - Temporal - Santa Cruz
Encargado(a) de Sucursal Montero
Jefe de Departamento Legal - Chuquisaca
Promotores(as) de Ventas - Cochabamba
Consignatario(a) para Administrar Tienda
Vendedores(as) - Sucre
Vendedores(as) - Tarija
Ingeniero(a) en Telecomunicaciones
Asistente de Licitaciones
Ejecutivo(a) de Ventas - Santa Cruz
Vendedor(a) - Santa Cruz de la Sierra
Supervisor(a) de Ventas Canal Varios - Potosí
Cajer

Ingeniero(a) Financiero(a)
Ejecutivo(a) Comercial (Sponsor) - Plazo Fijo
Pasante de Logística - Área de Carga - La Paz
Prospector(a) - Provincias
Auxiliar de Atención al Cliente y Tesorería - Tarija
Supervisor(a) de Bar
Arquitecto(a) Diseñador(a) con Perfil Comercial
Visitador(a) Médico(a)
Jefe de Administración - Santa Cruz
Oficial de Ventas - La Paz
Oficial de Ventas - Santa Cruz
Ingeniero(a) Electrónico(a) o Biomédico(a) para Servicio Técnico
Encargado(a) de Seguridad y Vigilancia
Recepcionista
Vendedor(a) - Canal Profesional
Merchandiser (Mercaderista) - La Paz
Promotor(a) de Ventas - La Paz
Jefe de Personal
Chofer de Distribución - Santa Cruz
Gerente Comercial - Cochabamba
Supervisor(a) Comercial
Experto(a) en Redacción de Contenido
Oficial de Negocios de Microempresa - Santa Cruz
Gerente de Zona - Santa Cruz
Pasante de Ingeniería Civil - Santa Cruz
Asesor(a) de Servicio
Pasante de Arquitectura - Santa Cruz
Ejecutivo(a) de Atención a Clientes - Minutas
Analista Control de Calidad 

Ayudante de Cocina
Ejecutivo(a) de Ventas para Canal Horizontal
Ejecutivo(a) de Ventas - Canal Mercados Mayoristas
Personal para Atención al Cliente en Sala VIP - El Alto
Encargado(a) de Tienda
Personal para Múltiples Tareas en Sala VIP - Santa Cruz
Auxiliar Contable
Agente de Turismo Receptivo
Jefe de Plataformas Digitales - Santa Cruz
Monitor(a) de Servicio Comercial - El Alto
Asistente de Facturación - El Alto
Operador(a) de Monitoreo - Cochabamba
Almacenero(a) - El Alto
Practicante para el Área de Transporte - El Alto
Encargado(a) de Monitoreo y Control de Rastreo Satelital
Encargado(a) de Vehículos
Encargado(a) Comercial (Sponsor)
Azafata - Santa Cruz
Abogado(a) Sénior - Santa Cruz
Ejecutivo(a) de Ventas - Santa Cruz
Ejecutivo(a) de Ventas - Cochabamba
Asesor(a) de Créditos - Caranavi
Asesor(a) de Créditos - Tupiza
Mensajero(a)
Oficial de Seguridad de la Información
Almaceneros(as) - La Paz
Responsable Administrativo(a) Comercial
Contador(a) Júnior
Asesor(a) de Viaje para Mayorist

In [14]:
len(Results)

758

In [15]:
import csv
# Open the file in the mode append and the encoding utf-8
outfile = open("/Users/Roberto/Documents/Python Scripts/Machine Learning Projects/Labor Market/TRABAJOPOLIS_FEB2019.csv", 'a',newline='', encoding = 'utf-8')
writer = csv.writer(outfile)
try:
    for i in range(len(Results)):
        csvRow = []
        # Transform the list shape:
        if Results[i][0] is not None:
            
            for cell in Results[i][0]:
                csvRow.append(cell)
                
            csvRow.append(Results[i][1])
            csvRow.append(Results[i][2])

            writer.writerow(csvRow)
finally:
    outfile.close() 

In [16]:
Results[400]

(['MAKIBER S.A. SUCURSAL BOLIVIA',
  '831182',
  'La Paz, Bolivia',
  'Otros ',
  'No Declarado ',
  '14 Febrero 2020',
  '15 Marzo 2020',
  'se requiereprofesional tecnico dibujanterequisitos disponibilidad de tiempo 8 horas continuas deseo de superacion personal y profesional titulado de la carrera de arquitectura o similares conocimiento de excel nivel medio no excluyente conocimiento en autocad  conocimiento en revit  experiencia minima como dibujante de un ano en proyectos de construccion se valorara experiencia y conocimiento en otros programas de diseno y dibujobeneficios contrato formal por todo el tiempo del proyecto de construccion se pagara sueldo y bonificacion mensualmente todos los beneficios de leyinteresados deben enviar sus hojas de vida indicando su pretension salarial por trabajopolisbo'],
 'Profesional Técnico Dibujante',
 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/831182/Profesional-Técnico-Dibujante.html?searchId=1546400657&page=1')

In [17]:
Results[0]

(['LUÁ S.R.L.',
  '839112',
  'Santa Cruz de la Sierra, Bolivia',
  'Contabilidad y Economía ',
  'No Declarado ',
  '02 Marzo 2020',
  '01 Abril 2020',
  'nuestro cliente lider en comercializacion y distribucion de productos farmaceuticos requiere incorporar un profesional para ocupar el cargo decontador de costosobjetivo del cargoresponsable de analizar validar y registrar los costos en los que incurre la empresa en el desarrollo de su actividad economica disenando sistemas de control de costos inventarios y activos fijos optimizando los procesos para apoyar a la toma de decisiones a nivel internorequisitos profesional formado en contabilidad publica o auditoria financiera con registro en el colegio de contadores yo auditores minimo 5 anos de experiencia en el cargo o similares en empresas del sector comercial o industrial  solidos conocimientos contables manejo de normas ifrs conocimientos tributarios actualizados estudios especializados en costos  manejo de sistemas informaticos in

In [18]:
Results = trabajopolis(['sql'])

The request response cycle was successful
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815585/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815434/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/815136/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/811069/Cajero(a)---Santa-Cruz.html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/810620/Cajero(a)---La-Paz.html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/810125/Cajero---Potosí.html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/809885/Cajero(a).html?searchId=1576530021.38&page=1
https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/809309/Cajero(a)-y-Atención-al-Cliente

NameError: name 'time' is not defined

In [ ]:
hola = Results[1000:]

In [ ]:
for i in range(len(hola)-1):
    csvRow = []
        
    if Results[i][0] is not None:
            
        for cell in hola[i][0]:
            csvRow.append(cell)
                
        csvRow.append(hola[i][1])
        csvRow.append(hola[i][2])

print(csvRow)

In [53]:
csvrow = []
for cell in Results[i][0]:
    #print(cell)
    csvrow.append(cell)
csvrow.append(Results[i][1])
csvrow.append(Results[i][2])
print(csvrow)

['Guayabochi srl', '699870', 'Santa Cruz de la Sierra, Bolivia', 'Administración y Oficina ', 'No Declarado ', '07 Enero 2019', '06 Febrero 2019', 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo', 'Mensajero(a)', 'https://www.trabajopolis.bo/oferta-de-trabajo-y-empleo-en-bolivia/699870/Mensajero(a).html?searchId=1546400657&page=1']


In [43]:
Results[i][0]

['Guayabochi srl',
 '699870',
 'Santa Cruz de la Sierra, Bolivia',
 'Administración y Oficina ',
 'No Declarado ',
 '07 Enero 2019',
 '06 Febrero 2019',
 'empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo']

In [44]:
for cell in Results[i][0]:
    print(cell)

Guayabochi srl
699870
Santa Cruz de la Sierra, Bolivia
Administración y Oficina 
No Declarado 
07 Enero 2019
06 Febrero 2019
empresa de servicios busca incorporar a su equipo de trabajo el cargomensajerorequisitos bachiller yo estudiante de carreras administrativas buen manejo de microsoft office buena ortografia y redaccion disponibilidad de viajes trabajo bajo presion disponibilidad inmediataofrecemos buen ambiente de trabajo remuneracion de acorde al cargolas personas que cumplan con el perfil mencionado postularse enviando su pretension salarial hoja de vida y fotografia actualizada por trabajopolisbo


In [45]:
csvRow.append(cell)